# CDX Tranche Pricing - Gaussian Copula Model

Implementation of one-factor Gaussian copula for CDX.NA.IG.45 tranche pricing.

In [1]:
import numpy as np
import pandas as pd
import json
from scipy.stats import norm
from scipy.optimize import minimize_scalar
import warnings
warnings.filterwarnings('ignore')

## 1. Load Market Data

In [ ]:
constituents = pd.read_csv('../data/cdx_constituents.csv').dropna(subset=['Company', '5Y_Spread'])
ois_curve = pd.read_csv('../data/ois_curve.csv').dropna(subset=['Tenor'])

with open('../data/cdx_market_data.json', 'r') as f:
    market_data = json.load(f)

print(f"Loaded {len(constituents)} companies")
print(f"CDS spread range: {constituents['5Y_Spread'].min():.2f} - {constituents['5Y_Spread'].max():.2f} bps")
print(f"Mean spread: {constituents['5Y_Spread'].mean():.2f} bps")

## 2. Bootstrap Survival Probabilities

In [3]:
def bootstrap_survival(spread_bps, recovery_rate, maturity=5.0):
    spread = spread_bps / 10000.0
    hazard_rate = spread / (1 - recovery_rate)
    time_points = np.arange(0, maturity + 0.25, 0.25)
    survival = np.exp(-hazard_rate * time_points)
    return survival

survival_curves = []
default_probs = []

for _, row in constituents.iterrows():
    surv = bootstrap_survival(row['5Y_Spread'], row['Recovery'])
    survival_curves.append(surv)
    default_probs.append(1 - surv[-1])

default_probs = np.array(default_probs)
weights = constituents['Weight'].values

print(f"5Y default probability range: {default_probs.min()*100:.2f}% - {default_probs.max()*100:.2f}%")
print(f"Portfolio avg 5Y PD: {np.sum(weights * default_probs)*100:.2f}%")

5Y default probability range: 1.86% - 13.11%
Portfolio avg 5Y PD: 4.63%


## 3. Gaussian Copula Model

In [4]:
class GaussianCopula:
    def __init__(self, rho, n_points=500):
        self.rho = rho
        self.Y_values = np.linspace(-5, 5, n_points)
        pdf = norm.pdf(self.Y_values)
        dy = self.Y_values[1] - self.Y_values[0]
        self.Y_weights = pdf * dy / (pdf * dy).sum()
    
    def conditional_pd(self, PD, Y):
        if PD <= 0 or PD >= 1:
            return max(0, min(1, PD))
        K = norm.ppf(PD)
        return norm.cdf((K - np.sqrt(self.rho) * Y) / np.sqrt(1 - self.rho))
    
    def portfolio_loss(self, pds, weights, recovery=0.40):
        lgd = 1 - recovery
        losses = np.zeros(len(self.Y_values))
        
        for i, (Y, prob) in enumerate(zip(self.Y_values, self.Y_weights)):
            cond_pds = np.array([self.conditional_pd(pd, Y) for pd in pds])
            losses[i] = lgd * np.sum(weights * cond_pds)
        
        return losses, self.Y_weights
    
    def tranche_el(self, losses, probs, attach, detach):
        width = detach - attach
        tranche_loss = np.maximum(0, np.minimum(losses - attach, width)) / width
        return np.sum(tranche_loss * probs)

## 4. Tranche Pricing

In [5]:
def price_tranches(copula, pds, weights, discount_rate=0.032325, maturity=5.0):
    losses, probs = copula.portfolio_loss(pds, weights)
    
    payment_dates = np.arange(0.25, maturity + 0.25, 0.25)
    df = np.exp(-discount_rate * payment_dates)
    avg_surv = 1 - np.sum(weights * pds)
    rpv01 = np.sum(df * avg_surv * 0.25)
    
    tranches = [
        ('equity_0_3', 0.00, 0.03, 500.0),
        ('mezz_3_7', 0.03, 0.07, None),
        ('mezz_7_10', 0.07, 0.10, None),
        ('senior_10_15', 0.10, 0.15, None),
        ('senior_15_100', 0.15, 1.00, None)
    ]
    
    results = {}
    for name, attach, detach, running in tranches:
        el = copula.tranche_el(losses, probs, attach, detach)
        spread = (el / rpv01) * 10000
        upfront = el - (running / 10000 * rpv01 if running else 0)
        
        results[name] = {
            'el': el * 100,
            'spread': spread,
            'upfront': upfront * 100 if running else None
        }
    
    return results

## 5. Calibration

In [ ]:
target_upfront = market_data['market_tranche_prices']['equity_0_3_upfront']
ois_5y = ois_curve[ois_curve['Tenor'] == '5Y']['Mid_Yield'].values[0]

def objective(rho):
    try:
        copula = GaussianCopula(rho, n_points=200)
        prices = price_tranches(copula, default_probs, weights, ois_5y)
        
        # Minimize total weighted error across all tranches
        total_error = 0
        weights_obj = [2.0, 1.0, 1.0, 1.0, 1.0]  # Weight equity more, but not exclusively
        
        # Equity tranche (upfront)
        model_upfront = prices['equity_0_3']['upfront']
        error_equity = ((model_upfront - target_upfront) / 100) ** 2
        total_error += weights_obj[0] * error_equity
        
        # Other tranches (spreads)
        market_spreads = [
            market_data['market_tranche_prices']['mezz_3_7'],
            market_data['market_tranche_prices']['mezz_7_10'],
            market_data['market_tranche_prices']['senior_10_15'],
            market_data['market_tranche_prices']['senior_15_100']
        ]
        
        model_spreads = [
            prices['mezz_3_7']['spread'],
            prices['mezz_7_10']['spread'],
            prices['senior_10_15']['spread'],
            prices['senior_15_100']['spread']
        ]
        
        for i, (model_spread, market_spread) in enumerate(zip(model_spreads, market_spreads)):
            error = ((model_spread - market_spread) / 100) ** 2  # Normalize to same scale
            total_error += weights_obj[i+1] * error
        
        return total_error
    except:
        return 1e10

result = minimize_scalar(objective, bounds=(0.05, 0.95), method='bounded')
rho_optimal = result.x

print(f"Calibrated correlation: {rho_optimal:.4f} ({rho_optimal*100:.2f}%)")
print(f"Target equity upfront: {target_upfront:.2f}%")

## 6. Final Pricing with Calibrated Model

In [7]:
copula_final = GaussianCopula(rho_optimal, n_points=500)
model_prices = price_tranches(copula_final, default_probs, weights, ois_5y)

market_prices = market_data['market_tranche_prices']

comparison = []
for name, model in model_prices.items():
    if name == 'equity_0_3':
        market_val = market_prices['equity_0_3_upfront']
        model_val = model['upfront']
        error = abs(model_val - market_val)
        unit = '%'
    else:
        market_val = market_prices[name]
        model_val = model['spread']
        error = abs(model_val - market_val)
        unit = 'bps'
    
    comparison.append({
        'Tranche': name,
        'Model': f"{model_val:.2f} {unit}",
        'Market': f"{market_val:.2f} {unit}",
        'Error': f"{error:.2f} {unit}"
    })

df_result = pd.DataFrame(comparison)
print("\nPricing Results:")
print(df_result.to_string(index=False))

errors = [abs(model_prices['equity_0_3']['upfront'] - market_prices['equity_0_3_upfront'])]
for name in ['mezz_3_7', 'mezz_7_10', 'senior_10_15', 'senior_15_100']:
    errors.append(abs(model_prices[name]['spread'] - market_prices[name]))

ape = sum(errors)
print(f"\nAbsolute Pricing Error (APE): {ape:.2f}")
print(f"Mean Absolute Error (MAE): {ape/len(errors):.2f}")


Pricing Results:
      Tranche      Model     Market      Error
   equity_0_3    28.29 %    28.30 %     0.01 %
     mezz_3_7 405.20 bps  95.08 bps 310.12 bps
    mezz_7_10 184.32 bps 113.10 bps  71.22 bps
 senior_10_15  87.34 bps  55.94 bps  31.40 bps
senior_15_100   3.44 bps  21.72 bps  18.28 bps

Absolute Pricing Error (APE): 431.03
Mean Absolute Error (MAE): 86.21


## 7. Correlation Skew Analysis

In [8]:
base_corr = market_data['base_correlations']

print("Market Base Correlations:")
for k, v in base_corr.items():
    print(f"  {k}: {v:.2f}%")

print(f"\nModel single correlation: {rho_optimal*100:.2f}%")
print(f"\nMarket correlation skew: {base_corr['detach_3']:.2f}% → {base_corr['detach_100']:.2f}%")
print(f"Spread: {base_corr['detach_100'] - base_corr['detach_3']:.2f} percentage points")
print("\nGaussian copula uses single flat correlation - cannot capture skew.")

Market Base Correlations:
  detach_3: 47.67%
  detach_7: 47.67%
  detach_10: 55.99%
  detach_15: 60.33%
  detach_100: 67.56%

Model single correlation: 33.67%

Market correlation skew: 47.67% → 67.56%
Spread: 19.90 percentage points

Gaussian copula uses single flat correlation - cannot capture skew.


In [ ]:
# Calculate model-implied base correlations for each tranche
def calculate_base_correlation(target_spread, attachment, detachment, is_equity=False):
    """
    Find the correlation that matches the market spread for a given tranche.
    This is the base correlation for that detachment point.
    """
    def objective(rho):
        copula = GaussianCopula(rho, n_points=200)
        losses, probs = copula.portfolio_loss(default_probs, weights)
        el = copula.tranche_el(losses, probs, attachment, detachment)
        
        if is_equity:
            # For equity, match upfront payment
            model_spread = (el / rpv01) * 10000
            model_upfront = el - (500 / 10000 * rpv01)
            return (model_upfront * 100 - target_spread) ** 2
        else:
            # For other tranches, match spread
            model_spread = (el / rpv01) * 10000
            return (model_spread - target_spread) ** 2
    
    try:
        result = minimize_scalar(objective, bounds=(0.01, 0.99), method='bounded')
        return result.x * 100  # Return as percentage
    except:
        return None

# Calculate base correlations for each detachment point
model_base_corr = {}

# Equity tranche (0-3%)
model_base_corr['detach_3'] = calculate_base_correlation(
    market_prices['equity_0_3_upfront'], 0.00, 0.03, is_equity=True
)

# Mezzanine tranches
model_base_corr['detach_7'] = calculate_base_correlation(
    market_prices['mezz_3_7'], 0.00, 0.07, is_equity=False
)

model_base_corr['detach_10'] = calculate_base_correlation(
    market_prices['mezz_7_10'], 0.00, 0.10, is_equity=False
)

# Senior tranches
model_base_corr['detach_15'] = calculate_base_correlation(
    market_prices['senior_10_15'], 0.00, 0.15, is_equity=False
)

model_base_corr['detach_100'] = calculate_base_correlation(
    market_prices['senior_15_100'], 0.00, 1.00, is_equity=False
)

print("\nModel-Implied Base Correlations:")
print("="*50)
print(f"{'Detachment':<15} {'Market':<15} {'Model':<15}")
print("-"*50)
for key in ['detach_3', 'detach_7', 'detach_10', 'detach_15', 'detach_100']:
    market_val = base_corr.get(key, 'N/A')
    model_val = model_base_corr.get(key)
    if model_val:
        print(f"{key:<15} {market_val:>6.2f}%        {model_val:>6.2f}%")
    else:
        print(f"{key:<15} {market_val:>6.2f}%        {'N/A':>7}")

# Calculate correlation skew
if model_base_corr['detach_3'] and model_base_corr['detach_100']:
    model_skew = model_base_corr['detach_100'] - model_base_corr['detach_3']
    market_skew = base_corr['detach_100'] - base_corr['detach_3']
    print(f"\nCorrelation Skew:")
    print(f"  Market: {market_skew:.2f} pp")
    print(f"  Model:  {model_skew:.2f} pp")
    print(f"  Difference: {abs(model_skew - market_skew):.2f} pp")

## 7.5 Calculate Model Base Correlations

## 8. Save Results

In [ ]:
results = {
    'calibrated_correlation': float(rho_optimal),
    'ape': float(ape),
    'mae': float(ape / len(errors)),
    'model_prices': {k: {'spread': float(v['spread']), 'upfront': float(v['upfront']) if v['upfront'] else None} 
                     for k, v in model_prices.items()}
}

with open('../results/results.json', 'w') as f:
    json.dump(results, f, indent=2)

df_result.to_csv('../results/pricing_comparison.csv', index=False)

print("Results saved to results.json and pricing_comparison.csv")